# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
T. Henning  ->  T. Henning  |  ['T. Henning']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
J. Liu  ->  J. Liu  |  ['J. Liu']
S. Li  ->  S. Li  |  ['S. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Arxiv has 62 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.15937
extracting tarball to tmp_2407.15937... done.


J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']


Found 89 bibliographic references in tmp_2407.15937/ms.bbl.
syntax error in line 193: '=' expected
Retrieving document from  https://arxiv.org/e-print/2407.15943
extracting tarball to tmp_2407.15943... done.


Found 64 bibliographic references in tmp_2407.15943/ghosh_ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.16461
extracting tarball to tmp_2407.16461...

...

K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


Found 91 bibliographic references in tmp_2407.16651/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.16654
extracting tarball to tmp_2407.16654...

 done.


J. Liu  ->  J. Liu  |  ['J. Liu']
S. Li  ->  S. Li  |  ['S. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']


Found 66 bibliographic references in tmp_2407.16654/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15937-b31b1b.svg)](https://arxiv.org/abs/2407.15937) | **Toward Exoplanet Transit Spectroscopy Using JWST/MIRI's Medium Resolution Spectrometer**  |
|| D. Deming, et al. -- incl., <mark>J. Bouwman</mark> |
|*Appeared on*| *2024-07-24*|
|*Comments*| *Accepted for PASP. 23 pages, 14 figures, 4 tables*|
|**Abstract**|            The Mid-Infrared Instrument (MIRI)'s Medium Resolution Spectrometer (the MRS) on JWST has potentially important advantages for transit and eclipse spectroscopy of exoplanets, including lack of saturation for bright host stars, wavelength span to longward of 20 microns, and JWST's highest spectral resolving power. We here test the performance of the MRS for time series spectroscopy by observing the secondary eclipse of the bright stellar eclipsing binary R Canis Majoris. Our observations push the MRS into saturation at the shortest wavelength, more than for any currently known exoplanet system. We find strong charge migration between pixels that we mitigate using a custom data analysis pipeline. Our data analysis recovers much of the spatial charge migration by combining detector pixels at the group level, via weighting by the point spread function. We achieve nearly photon-limited performance in time series data at wavelengths longward of 5.2 microns. In 2017, Snellen et al. suggested that the MRS could be used to detect carbon dioxide absorption from the atmosphere of the temperate planet orbiting Proxima Centauri. We infer that the relative spectral response of the MRS versus wavelength is sufficiently stable to make that detection feasible. As regards the secondary eclipse of this Algol-type binary, we measure the eclipse depth by summing our spectra over the wavelengths in four channels, and also measuring the eclipse depth as observed by TESS. Those eclipse depths require a temperature for the secondary star that is significantly hotter than previous observations in the optical to near-IR, probably due to irradiation by the primary star. At full spectral resolution of the MRS, we find atomic hydrogen recombination emission lines in the secondary star, from principal quantum levels n = 7, 8, 10, and 14.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15943-b31b1b.svg)](https://arxiv.org/abs/2407.15943) | **Closing the gap: secular evolution of bar-induced dark gaps in presence of thick discs**  |
|| <mark>S. Ghosh</mark>, et al. |
|*Appeared on*| *2024-07-24*|
|*Comments*| *13 pages, 12 figures, 1 table (including appendix), accepted for publication in MNRAS*|
|**Abstract**|            The presence of dark gaps, a preferential light deficit along the bar minor axis, is observationally well known. The properties of dark gaps are thought to be associated with the properties of bars, and their spatial locations are often associated with bar resonances. However, a systematic study, testing the robustness and universality of these assumptions, is still largely missing. Here, we investigate the formation and evolution of bar-induced dark gaps using a suite of N-body models of (kinematically cold) thin and (kinematically hot) thick discs with varying thick disc mass fraction, and different thin-to-thick disc geometry. We find that dark gaps are a natural consequence of the trapping of disc stars by the bar. The properties of dark gaps (such as strength and extent) are well correlated with the properties of bars. For stronger dark gaps, the fractional mass loss along the bar minor axis can reach up to ~60-80 percent of the initial mass contained, which is redistributed within the bar. These trends hold true irrespective of the mass fraction in the thick disc and the assumed disc geometry. In all our models harbouring slow bars, none of the resonances (corotation, Inner Lindblad resonance, and 4:1 ultra-harmonic resonance) associated with the bar correspond to the location of dark gaps, thereby suggesting that the location of dark gaps is not a universal proxy for these bar resonances, in contrast with earlier studies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.16461-b31b1b.svg)](https://arxiv.org/abs/2407.16461) | **Revisiting the dynamical masses of the transiting planets in the young AU Mic system: Potential AU Mic b inflation at $\sim$20 Myr**  |
|| M. Mallorquín, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-24*|
|*Comments*| *Accepted in A&A. 28 pages, 15 figures*|
|**Abstract**|            Understanding planet formation is important in the context of the origin of planetary systems in general and of the Solar System in particular, as well as to predict the likelihood of finding Jupiter, Neptune, and Earth analogues around other stars. We aim to precisely determine the radii and dynamical masses of transiting planets orbiting the young M star AU Mic using public photometric and spectroscopic datasets. We characterise the stellar activity and physical properties (radius, mass, density) of the transiting planets in the young AU Mic system through joint transit and radial velocity fits with Gaussian processes. We determine a radius of $R^{b}$= 4.79 +/- 0.29 R$_\oplus$, a mass of $M^{b}$= 9.0 +/- 2.7 M$_\oplus$, and a bulk density of $\rho^{b}$ = 0.49 +/- 0.16 g cm$^{-3}$ for the innermost transiting planet AU Mic b. For the second known transiting planet, AU Mic c, we infer a radius of $R^{c}$= 2.79 +/- 0.18 R$_\oplus$, a mass of $M^{c}$= 14.5 +/- 3.4 M$_\oplus$, and a bulk density of $\rho^{c}$ = 3.90 +/- 1.17 g cm$^{-3}$. According to theoretical models, AU Mic b may harbour an H2 envelope larger than 5\% by mass, with a fraction of rock and a fraction of water. AU Mic c could be made of rock and/or water and may have an H2 atmosphere comprising at most 5\% of its mass. AU Mic b has retained most of its atmosphere but might lose it over tens of millions of years due to the strong stellar radiation, while AU Mic c likely suffers much less photo-evaporation because it lies at a larger separation from its host. Using all the datasets in hand, we determine a 3$\sigma$ upper mass limit of $M^{[d]}\sin{i}$ = 8.6 M$_{\oplus}$ for the AU Mic 'd' TTV-candidate. In addition, we do not confirm the recently proposed existence of the planet candidate AU Mic 'e' with an orbital period of 33.4 days.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.16578-b31b1b.svg)](https://arxiv.org/abs/2407.16578) | **Unveiling In-Situ Spheroid Formation in Distant, Submillimeter-Bright Galaxies**  |
|| Q.-H. Tan, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-07-24*|
|*Comments*| *Resubmitted to Nature after replying to the first round of referee reports*|
|**Abstract**|            The majority of stars in today's Universe reside within spheroids, which are bulges of spiral galaxies and elliptical galaxies. Their formation is still an unsolved problem. Infrared/submm-bright galaxies at high redshifts have long been suspected to be related to spheroids formation. Proving this connection has been hampered so far by heavy dust obscuration when focusing on their stellar emission or by methodologies and limited signal-to-noise ratios when looking at submm wavelengths. Here we show that spheroids are directly generated by star formation within the cores of highly luminous starburst galaxies in the distant Universe. This follows from the ALMA submillimeter surface brightness profiles which deviate significantly from those of exponential disks, and from the skewed-high axis-ratio distribution, both derived with a novel analysis technique. These galaxies are fully triaxial rather than flat disks: scale-height ratios are 0.5 on average and $>0.6$ for most spatially compact systems. These observations, supported by simulations, reveal a cosmologically relevant pathway for in-situ spheroid formation through starbursts likely preferentially triggered by interactions (and mergers) acting on galaxies fed by non co-planar gas accretion streams.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.16651-b31b1b.svg)](https://arxiv.org/abs/2407.16651) | **Hints of planet formation signatures in a large-cavity disk studied in the AGE-PRO ALMA Large Program**  |
|| A. Sierra, et al. -- incl., <mark>K. Schwarz</mark> |
|*Appeared on*| *2024-07-24*|
|*Comments*| *24 pages, 15 figures*|
|**Abstract**|            Detecting planet signatures in protoplanetary disks is fundamental to understanding how and where planets form. In this work, we report dust and gas observational hints of planet formation in the disk around 2MASS-J16120668-301027, as part of the ALMA Large Program ``AGE-PRO: ALMA survey of Gas Evolution in Protoplanetary disks''. The disk was imaged with the Atacama Large Millimeter/submillimeter Array (ALMA) at Band 6 (1.3 mm) in dust continuum emission and four molecular lines: $^{12}$CO(J=2-1), $^{13}$CO(J=2-1), C$^{18}$O(J=2-1), and H$_2$CO(J=3$_{(3,0)}$-2$_{(2,0)}$). Resolved observations of the dust continuum emission (angular resolution of $\sim 150$ mas, 20 au) show a ring-like structure with a peak at $0.57 ^{\prime \prime}$ (75 au), a deep gap with a minimum at 0.24$^{\prime \prime}$ (31 au), an inner disk, a bridge connecting the inner disk and the outer ring, along with a spiral arm structure, and a tentative detection (to $3\sigma$) of a compact emission at the center of the disk gap, with an estimated dust mass of $\sim 2.7-12.9$ Lunar masses. We also detected a kinematic kink (not coincident with any dust substructure) through several $^{12}$CO channel maps (angular resolution $\sim$ 200 mas, 30 au), located at a radius of $\sim 0.875^{\prime \prime}$ (115.6 au). After modeling the $^{12}$CO velocity rotation around the protostar, we identified a purple tentative rotating-like structure at the kink location with a geometry similar to that of the disk. We discuss potential explanations for the dust and gas substructures observed in the disk, and their potential connection to signatures of planet formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.16654-b31b1b.svg)](https://arxiv.org/abs/2407.16654) | **Magnetic Fields in Massive Star-forming Regions (MagMaR): Unveiling an Hourglass Magnetic Field in G333.46-0.16 using ALMA**  |
|| P. Saha, et al. -- incl., <mark>J. Liu</mark>, <mark>S. Li</mark>, <mark>H. Beuther</mark> |
|*Appeared on*| *2024-07-24*|
|*Comments*| **|
|**Abstract**|            The contribution of the magnetic field to the formation of high-mass stars is poorly understood. We report the high-angular resolution ($\sim0.3^{\prime\prime}$, 870 au) map of the magnetic field projected on the plane of the sky (B$_\mathrm{POS}$) towards the high-mass star forming region G333.46$-$0.16 (G333), obtained with the Atacama Large Millimeter/submillimeter Array (ALMA) at 1.2 mm as part of the Magnetic Fields in Massive Star-forming Regions (MagMaR) survey. The B$_\mathrm{POS}$ morphology found in this region is consistent with a canonical ``hourglass'' which suggest a dynamically important field. This region is fragmented into two protostars separated by $\sim1740$ au. Interestingly, by analysing H$^{13}$CO$^{+}$ ($J=3-2$) line emission, we find no velocity gradient over the extend of the continuum which is consistent with a strong field. We model the B$_\mathrm{POS}$, obtaining a marginally supercritical mass-to-flux ratio of 1.43, suggesting an initially strongly magnetized environment. Based on the Davis-Chandrasekhar-Fermi method, the magnetic field strength towards G333 is estimated to be 5.7 mG. The absence of strong rotation and outflows towards the central region of G333 suggests strong magnetic braking, consistent with a highly magnetized environment. Our study shows that despite being a strong regulator, the magnetic energy fails to prevent the process of fragmentation, as revealed by the formation of the two protostars in the central region.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.15937/./first_group_plot.png', 'tmp_2407.15937/./results_1_4.png', 'tmp_2407.15937/./results_2_3.png', 'tmp_2407.15937/./fringe_fig.png']
copying  tmp_2407.15937/./first_group_plot.png to _build/html/
copying  tmp_2407.15937/./results_1_4.png to _build/html/
copying  tmp_2407.15937/./results_2_3.png to _build/html/
copying  tmp_2407.15937/./fringe_fig.png to _build/html/
exported in  _build/html/2407.15937.md
    + _build/html/tmp_2407.15937/./first_group_plot.png
    + _build/html/tmp_2407.15937/./results_1_4.png
    + _build/html/tmp_2407.15937/./results_2_3.png
    + _build/html/tmp_2407.15937/./fringe_fig.png
found figures ['tmp_2407.15943/./collage_darklanes_endstep_for_paper_ML12.png', 'tmp_2407.15943/./plot_mass_loss_fran_style_rthickS01.png', 'tmp_2407.15943/./circular_velocity_patternspeed_example.png']
copying  tmp_2407.15943/./collage_darklanes_endstep_for_paper_ML12.png to _build/html/
copying  tmp_2407.15943/./plot_mass_loss_fran_style_rthickS01.pn

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Toward Exoplanet Transit Spectroscopy Using JWST/MIRI's Medium Resolution Spectrometer

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.15937-b31b1b.svg)](https://arxiv.org/abs/2407.15937)<mark>Appeared on: 2024-07-24</mark> -  _Accepted for PASP. 23 pages, 14 figures, 4 tables_

</div>
<div id="authors">

D. Deming, et al. -- incl., <mark>J. Bouwman</mark>

</div>
<div id="abstract">

**Abstract:** The Mid-Infrared Instrument (MIRI)'s Medium Resolution Spectrometer (the MRS) on JWST has potentially important advantages for transit and eclipse spectroscopy of exoplanets, including lack of saturation for bright host stars, wavelength span to longward of 20 microns, and JWST's highest spectral resolving power.  We here test the performance of the MRS for time series spectroscopy by observing the secondary eclipse of the bright stellar eclipsing binary R Canis Majoris.  Our observations push the MRS into saturation at the shortest wavelength, more than for any currently known exoplanet system.  We find strong charge migration between pixels that we mitigate using a custom data analysis pipeline.  Our data analysis recovers much of the spatial charge migration by combining detector pixels at the group level, via weighting by the point spread function.  We achieve nearly photon-limited performance in time series data at wavelengths longward of 5.2 microns.  In 2017, Snellen et al. suggested that the MRS could be used to detect carbon dioxide absorption from the atmosphere of the temperate planet orbiting Proxima Centauri.  We infer that the relative spectral response of the MRS versus wavelength is sufficiently stable to make that detection feasible.  As regards the secondary eclipse of this Algol-type binary, we measure the eclipse depth by summing our spectra over the wavelengths in four channels, and also measuring the eclipse depth as observed by TESS.  Those eclipse depths require a temperature for the secondary star that is significantly hotter than previous observations in the optical to near-IR, probably due to irradiation by the primary star.  At full spectral resolution of the MRS, we find atomic hydrogen recombination emission lines in the secondary star, from principal quantum levels $n=$ 7, 8, 10, and 14.

</div>

<div id="div_fig1">

<img src="tmp_2407.15937/./first_group_plot.png" alt="Fig2" width="100%"/>

**Figure 2. -** Examples of non-ideal ramps in the R CMa data, using the brightest portion of the spectrum (channel 1 in grating position A/SHORT).  The points in blue, green, and red are real data points, but are plotted in color to call attention to the effects we discuss.  Panels 'a' and 'b' show jumps in the data.  The blue points mark a positive jump due to a cosmic ray hit.  The green points in panels a and b show negative jumps. Panel c shows the brighter-fatter effect; the red points mark an increase in slope due to charge migration from an adjoining pixel.  The Y-axes are in data numbers (DNs) divided by 10,000; read the left axis for panels a and b, panel c axis on the right. (*fig: first_group*)

</div>
<div id="div_fig2">

<img src="tmp_2407.15937/./results_1_4.png" alt="Fig11.1" width="50%"/><img src="tmp_2407.15937/./results_2_3.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Left: Secondary eclipse spectrum of R CMa in channel-1 (bottom) and channel-4 (top).  The portion of the channel-1 spectrum plotted in gray is deemed unreliable due to saturation (see \S\ref{sec: saturation}). Right: Eclipse spectrum in channels 2 (bottom) and 3 (top).  All spectra are plotted at single-pixel resolution (light blue), and smoothed over 10-pixels (dark blue).  Several hydrogen recombination emission lines are marked, with their upper and lower levels noted.  For example, the n=7 to n=6 line of atomic hydrogen is seen prominently at 12.37 $\mu$m.  See text for discussion of the spectral features. (*fig: spectra_hydrogen*)

</div>
<div id="div_fig3">

<img src="tmp_2407.15937/./fringe_fig.png" alt="Fig12" width="100%"/>

**Figure 12. -** Spectral fringes in the MRS spectra of R CMa.  _ Top panel (wavelength scale at top):_ Average of all integrations for a single spatial slice at both ingress and egress. Note the similarity of the fringe pattern for those two visits, that are separated by 18 days. _ Middle panel (wavelength scale at bottom):_ Expansion of the fringes from the top panel in both wavelength and electron number, and normalization to equal signals, so that the similarity of the fringes at both visits is more apparent.  _ Bottom panel (wavelength scale at bottom):_ Spectrum of a randomly selected single integration during ingress, with the defringed spectrum for that integration overplotted. (*fig: fringes*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.15937"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\mbox{\>{\rm km  s^{-1}}}}$
$\newcommand{\pc}{\>{\rm pc}}$
$\newcommand{\kpc}{\mbox{\>{\rm kpc}}}$
$\newcommand{\kkms}{\mbox{\>{\rm kpc  km  s^{-1}}}}$
$\newcommand{\kmsk}{\mbox{\>{\rm km  s^{-1}  kpc^{-1}}}}$
$\newcommand{\Gyr}{\mbox{\>{\rm Gyr}}}$
$\newcommand{\Myr}{\mbox{\>{\rm Myr}}}$
$\newcommand{\yr}{\mbox{\>{\rm yr}}}$
$\newcommand{\Msun}{\>{\rm M_{\odot}}}$
$\newcommand{\Rd}{\mbox{R_{\rm d}}}$
$\newcommand{\zd}{\mbox{z_{\rm d}}}$
$\newcommand{\md}{\mbox{M_{\rm d}}}$
$\newcommand{\vgsr}{\mbox{V_{\rm GSR}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\avg}[1]{\mbox{\left<{#1}\right>}}$
$\newcommand{\sig}[1]{\mbox{\sigma_{#1}}}$
$\newcommand{\feh}{\mbox{\rm[Fe/H]}}$
$\newcommand{\al}{\mbox{\rm \alpha}}$
$\newcommand{\alfe}{\mbox{\rm[O/Fe]}}$
$\newcommand{\tf}{\mbox{\tau}}$
$\newcommand{\rs}{\mbox{R_s}}$
$\newcommand{\abar}{\mbox{A_{\rm bar}}}$
$\newcommand{\vb}{\mbox{V_{\rm breath}}}$
$\newcommand{\vbend}{\mbox{V_{\rm bend}}}$
$\newcommand$
$\newcommand{\update}[1]{\textbf{\color{cyan} #1}}$</div>



<div id="title">

# Closing the gap: secular evolution of bar-induced dark gaps in presence of thick discs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.15943-b31b1b.svg)](https://arxiv.org/abs/2407.15943)<mark>Appeared on: 2024-07-24</mark> -  _13 pages, 12 figures, 1 table (including appendix), accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>S. Ghosh</mark>, et al.

</div>
<div id="abstract">

**Abstract:** The presence of dark gaps, a preferential light deficit along the bar minor axis, is observationally well known. The properties of dark gaps are thought to be associated with the properties of bars, and their spatial locations are often associated with bar resonances. However, a systematic study, testing the robustness and universality of these assumptions, is still largely missing.  Here, we investigate the formation and evolution of bar-induced dark gaps using a suite of $N$ -body models of (kinematically cold) thin and (kinematically hot) thick discs with varying thick disc mass fraction, and different thin-to-thick disc geometry. We find that dark gaps are a natural consequence of the trapping of disc stars by the bar. The properties of dark gaps (such as strength and extent) are well correlated with the properties of bars. For stronger dark gaps, the fractional mass loss along the bar minor axis can reach up to $\sim 60-80$ percent of the initial mass contained, which is redistributed within the bar. These trends hold true irrespective of the mass fraction in the thick disc and the assumed disc geometry.In all our models harbouring slow bars, none of the resonances (corotation, Inner Lindblad resonance, and 4:1 ultra-harmonic resonance) associated with the bar correspond to the location of dark gaps, thereby suggesting that the location of dark gaps is not a _universal proxy_ for these bar resonances, in contrast with earlier studies.

</div>

<div id="div_fig1">

<img src="tmp_2407.15943/./collage_darklanes_endstep_for_paper_ML12.png" alt="Fig11" width="100%"/>

**Figure 11. -** Face-on surface brightness distribution, calculated at the end of the simulation run ($t = 9 $\Gyr$$), for all thin+thick models considered here.
Black solid lines denote the contours of constant surface brightness. For each case, the bar is placed along the $x$-axis. The magenta and the blue dashed lines denote the bar major and minor axis, respectively. The black dashed circle denotes the location of maximum brightness contrast ($\Delta \mu_{\rm max}$), for details see the text. _Left panels_ show the surface brightness distribution for the rthickS models whereas  _middle panels_ and _right panels_ show the surface brightness distribution for the rthickE  and rthickG models, respectively. The thick disc fraction ($f_{\rm thick}$) varies from 0.1 to 0.9 (top to bottom), as indicated in the left-most panel of each row. A magnitude zero-point ($m_0$) of $22.5$ mag arcsec$^{-2}$ and  $\Upsilon_{T}/\Upsilon_{t}$ =1.2 are used to create the surface brightness from the intrinsic particle distribution. Here, 1 arcsec = $1 $\kpc$$. (*fig:densmap_endsteps_appendix*)

</div>
<div id="div_fig2">

<img src="tmp_2407.15943/./plot_mass_loss_fran_style_rthickS01.png" alt="Fig1" width="100%"/>

**Figure 1. -** Fractional mass loss, at different spatial locations along the bar minor axis, $\delta M_{*} (y_{\rm minor}, t)$(Eq. \ref{eq:mass_loss_FranStyle}), as a function of time for the thin+thick model rthickS0.1. The colour bar shows the spatial locations along the bar minor axis. The vertical dash-dotted line denotes the epoch of bar formation, $\tau_{\rm bar}$. $1 < y_{\rm minor}/ $\kpc$ < 5$ denotes the region of the dark gap. For details, see section \ref{sec:mass_loss_growth}. (*fig:mass_loss_along_minorAxis_FranStyle*)

</div>
<div id="div_fig3">

<img src="tmp_2407.15943/./circular_velocity_patternspeed_example.png" alt="Fig2" width="100%"/>

**Figure 2. -** Radial variation of the circular frequency ($\Omega$), epicyclic frequency ($\kappa$), $\Omega -\kappa/2$, and $\Omega- \kappa/4$(see the legend), calculated at $t = 1.35 $\Gyr$$ for the model rthickS0.1. The horizontal dash-dotted black line denotes the bar pattern speed ($\Omega_{\rm bar}$) at that epoch while the vertical cyan line denotes the location of the dark gap ($R_{\rm DG}$) at the same epoch. (*fig:circularVel_example*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.15943"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$
$\newcommand\hyper{@linkstart##1##2 $
$}$</div>



<div id="title">

# Revisiting the dynamical masses of the transiting planets in the young AU Mic system: Potential AU Mic b inflation at $\sim$20 Myr$\thanks{Full Tables \ref{tab:rv_CARMV}, \ref{tab:rv_CARMN}, \ref{tab:rv_HARPS}, and \ref{tab:rv_SPIRou} are only available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via \url{http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/}}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.16461-b31b1b.svg)](https://arxiv.org/abs/2407.16461)<mark>Appeared on: 2024-07-24</mark> -  _Accepted in A&A. 28 pages, 15 figures_

</div>
<div id="authors">

M. Mallorquín, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Understanding planet formation is important in the context of the origin of planetary systems in general and of the Solar System in particular, as well as to predict the likelihood of finding Jupiter, Neptune, and Earth analogues around other stars. We aim to precisely determine the radii and dynamical masses of transiting planets orbiting the young M star AU Mic using public photometric and spectroscopic datasets. We performed a joint fit analysis of the TESS and CHEOPS light curves and more than 400 high-resolution spectra collected with several telescopes and instruments. We characterise the stellar activity and physical properties (radius, mass, density) of the transiting planets in the young AU Mic system through joint transit and radial velocity fits with Gaussian processes. We determine a radius of $R_{p}^{b}$ = 4.79 $\pm$ 0.29 R $_\oplus$ , a mass of $M_{p}^{b}$ = 9.0 $\pm$ 2.7 M $_\oplus$ , and a bulk density of $\rho_{p}^{b}$ = 0.49 $\pm$ 0.16 g cm $^{-3}$ for the innermost transiting planet AU Mic b. For the second known transiting planet, AU Mic c, we infer a radius of $R_{p}^{c}$ = 2.79 $\pm$ 0.18 R $_\oplus$ , a mass of $M_{p}^{c}$ = 14.5 $\pm$ 3.4 M $_\oplus$ , and a bulk density of $\rho_{p}^{c}$ = 3.90 $\pm$ 1.17 g cm $^{-3}$ . According to theoretical models, AU Mic b may harbour an H $_{2}$ envelope larger than 5 \% by mass, with a fraction of rock and a fraction of water. AU Mic c could be made of rock and/or water and may have an H $_{2}$ atmosphere comprising at most 5 \% of its mass. AU Mic b has retained most of its atmosphere but might lose it over tens of millions of years due to the strong stellar radiation, while AU Mic c likely suffers much less photo-evaporation because it lies at a larger separation from its host. Using all the datasets in hand, we determine a 3 $\sigma$ upper mass limit of $M_{p}^{[d]}\sin{i}$ = 8.6 M $_{\oplus}$ for the AU Mic 'd' TTV-candidate. In addition, we do not confirm the recently proposed existence of the planet candidate AU Mic 'e' with an orbital period of 33.4 days. We investigated the level of the radial velocity variations and show that it is lower at longer wavelength with smaller changes from one observational campaign to another.

</div>

<div id="div_fig1">

<img src="tmp_2407.16461/./figures/AU_Mic_2pl_PH.png" alt="Fig10" width="100%"/>

**Figure 10. -** TESS light curve of AU Mic. The PDCSAP flux used in the analysis is shown as blue dots. The data considered as flares or transits are shown as red dots. The black and grey lines represent the stellar activity models for the stellar rotation and for the flares, respectively. The flare model was vertical shifted for a better visualisation. The vertical orange and purple bands indicate the timing of the planetary transits for AU Mic b and c, respectively. (*fig:LC_TESS*)

</div>
<div id="div_fig2">

<img src="tmp_2407.16461/./figures/SED.png" alt="Fig1" width="100%"/>

**Figure 1. -** Photometric SED of AU Mic (coloured dots) covers 0.13 $\mu$m to 670 $\mu$m. The photospheric emission of a 3600 K dwarf with solar metallicity is shown by the blue line \citep[BT-Settl model;][]{Allard2012}. AU Mic shows significant flux excesses at wavelengths shorter than 0.3 $\mu$m and longer than 40 $\mu$m, which are compatible with stellar activity and the presence of a circumstellar debris disc \citep{Kalas2004}, respectively. Horizontal error bars account for the width of the filter passbands. Effective wavelengths and widths of the passbands are taken from the Virtual Observatory filter database \citep{bayo2008}. (*fig:SED*)

</div>
<div id="div_fig3">

<img src="tmp_2407.16461/./figures/AU_Mic_2pl_PH_phased-folded_b.png" alt="Fig5.1" width="50%"/><img src="tmp_2407.16461/./figures/AU_Mic_2pl_PH_phased-folded_c.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Phase-folded light curves around transits of AU Mic b (top panel) and AU Mic c (bottom panel) for the alternative transit model. In each sub-panel, the TESS (blue dots) and CHEOPS data (orange dots) are shown, along with the binned data (white dots), the best TESS transit-fit model (black line) in the top, and the residuals from the best-fit in the bottom.
 (*fig:PH_folded*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.16461"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\textbi}[1]{\textbf{\textit{#1}}}$
$\newcommand{\mdash}{-}$
$\newcommand{\sime}{\sim}$
$\newcommand$
$\newcommand$
$\newcommand{\sersic}{S{\rm \acute{e}}rsic}$</div>



<div id="title">

# Unveiling In-Situ Spheroid Formation in Distant, Submillimeter-Bright Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.16578-b31b1b.svg)](https://arxiv.org/abs/2407.16578)<mark>Appeared on: 2024-07-24</mark> -  _Resubmitted to Nature after replying to the first round of referee reports_

</div>
<div id="authors">

Q.-H. Tan, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** **The majority of stars in today's Universe reside within spheroids, which are bulges of spiral galaxies and elliptical galaxies Baldry2006,  Gadotti2009**.Their formation is still an unsolved problem\cite{Brooks2016, Oser2012, Peng2010. Infrared/submm-bright galaxies at high redshiftsBlain2002 have long been suspected to be related to spheroids formationLilly1999, Archibald2002,  Dunne2003, DeLucia2006, Tacconi2008, Brisbin2017. Proving this connection has been hampered so far by heavy dust obscuration when focusing on their stellar emissionLeBail2023, Cardona-Torres2023, Gillman2023 or by  methodologies and limited signal-to-noise ratios when looking at  submm wavelengthsGullberg2019, Hodge2019.Here we show that spheroids are directly generated by star formation within the cores of highly luminous starburst galaxies in the distant Universe. This follows from the ALMA submillimeter surface brightness profiles which deviate significantly from those of exponential disks, and from the skewed-high axis-ratio distribution, both derived with a novel analysis technique. These galaxies are fully triaxial rather than flat disks: scale-height ratios are 0.5 on average and $>0.6$ for  most spatially compact systems. These observations, supported by simulations, reveal a cosmologically relevant pathway for in-situ spheroid formation through  starbursts likely preferentially triggered by interactions (and mergers) acting on galaxies fed by non co-planar gas accretion streams.}

</div>

<div id="div_fig1">

<img src="tmp_2407.16578/./reff_hist_plot.png" alt="Fig1.1" width="16%"/><img src="tmp_2407.16578/./nu_Re_scatter.png" alt="Fig1.2" width="16%"/><img src="tmp_2407.16578/./axis_hist_plot.png" alt="Fig1.3" width="16%"/><img src="tmp_2407.16578/./q_Re_scatter.png" alt="Fig1.4" width="16%"/><img src="tmp_2407.16578/./nu_model_hist_full.png" alt="Fig1.5" width="16%"/><img src="tmp_2407.16578/./nu_q_scatter.png" alt="Fig1.6" width="16%"/>

**Figure 1. -** \small**Physical properties of submm-bright galaxies,  measured from fitting Spergel light profiles to the visibilities.****Left**: distributions of measured parameters. **a**, half-light radius $R_{\rm e}$. **c**, comparison of the distributions of the observed axis ratio _q_ between our sample of galaxies (green) and local (U)LIRGs (purple). As reference, we plot the expected _q_ distributions for disks with a scale-height (C/A) of 0.1 (blue curve) and spheroids with a scale-height of 0.7 (red curve). **e**, Spergel index $\nu$. The blue curves in panel **e** represent the intrinsic distribution of $\nu$, which was modeled using a combination of two Gaussian functions, while the orange curve represents the best-fit $\nu$ distribution of sample galaxies by taking into account the uncertainty of $\nu$ measurement $\sigma_\nu$(see Methods). The error bar in each bin corresponds to the 1$\sigma$ Poisson error.  The vertical dashed lines in panels **a,c** represent the median value of the distribution and the horizontal bar indicates the error on the median, respectively. **Right**: relationships between measured parameters. **b**,  $R_{\rm e}$ versus Spergel index $\nu$. **d**, _q_ versus $R_{\rm e}$. **f**, _q_ versus $\nu$. **b,d,f**, data points are colour coded by _q_, $\nu$, and $R_{\rm e}$, respectively. The dashed lines in panels **b,e,f** mark the  $\nu=0$ threshold above which we classify galaxies as pure disks. **b,f**, the $\sersic$  indices listed on the top axis were converted using Eq. (\ref{eq:conversion}) (see Methods),  assuming $R_{\rm e}/\theta_{\rm b}=0.42$, where $\theta_{\rm b}$ is the beam size of the observations. This value is the median measured for the whole sample of galaxies. (*fig:histogram*)

</div>
<div id="div_fig2">

<img src="tmp_2407.16578/./axis_hist_model_full.png" alt="Fig2.1" width="16%"/><img src="tmp_2407.16578/./axis_BA_CA_scatter_full.png" alt="Fig2.2" width="16%"/><img src="tmp_2407.16578/./axis_hist_model_compact.png" alt="Fig2.3" width="16%"/><img src="tmp_2407.16578/./axis_BA_CA_scatter_compact.png" alt="Fig2.4" width="16%"/><img src="tmp_2407.16578/./axis_hist_model_extended.png" alt="Fig2.5" width="16%"/><img src="tmp_2407.16578/./axis_BA_CA_scatter_extended.png" alt="Fig2.6" width="16%"/>

**Figure 2. -** \small**Observed and 3$D$ intrinsic axis ratios for ALMA submm-bright galaxies.****Left**: distribution of observed axis ratio _q_.  **Right**: distribution of average intrinsic face-on axial ratio $B/A$ versus edge-on axial ratio $C/A$(scale-height), calculated as the first moment of the distribution of B/A and C/A, respectively. The sample distribution of $B/A$ is log-normal, while $C/A$ is Gaussian. **a,b**: the full sample. **c,d**: a subsample of $\Sigma_{\rm SFR}$-compact galaxies. **e,f**: a subsample of $\Sigma_{\rm SFR}$-extended galaxies. The red curves in left panels represent the MAP best-fit model with the highest posterior probability, which is identified as the step with highest posterior probability and nearest to the position defined by averages $\langle B/A \rangle$ and $\langle C/A \rangle$(the black squares in the right panels; see Methods). The error bar in each histogram bin corresponds to the 1$\sigma$ Poisson error. The dotted curve in panel **c** represents the expected distribution of projected axis ratios for galaxies with a spherical shape (C/A=1). The data points in right panels are colour coded by the posterior probability. Contours are plotted based on the density of the data points, using the posterior probability as weights. Contour levels are 1$\sigma$, 2$\sigma$, and 3$\sigma$. The black squares and error bars in the right panels represent the weighted mean and standard deviation derived from the histograms. We used the chi-square statistic to quantify the difference between the cumulative distribution function (CDF) of observational data and that of the best-fit model. The full sample and the two subsamples of $\Sigma_{\rm SFR}$-compact and -extended galaxies have $\chi^2 = $3.8, 5.8, and 9.2, respectively.
 (*fig:hist-axis*)

</div>
<div id="div_fig3">

<img src="tmp_2407.16578/./Eagle_data.png" alt="Fig14.1" width="50%"/><img src="tmp_2407.16578/./Submm_descendants_analysis_v2.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** \small**Merger history of submm galaxies using EAGLE cosmological simulation.****a**, Stellar mass and redshift distribution of the submm galaxy sample (orange symbols). **b**, SFR of the submm sample (shown in orange symbols) relative to the star-forming Main Sequence of the simulations  (blue solid line; all trends rescaled in star-formation to $z=1$, for clarity). Dark grey shaded regions mark the 16-84th percentiles of the relation, whereas light grey shaded regions indicate the 5-95th percentiles. Individual SFRs of the submm sample have been renormalized (see text). **c**, fraction of submm galaxies that underwent a major (blue symbols) or minor (grey symbols) mergers. The panel shows that major mergers do not seem to dominate the assembly of submm-bright galaxies. **d**, gas accretion rates of galaxies before they became submm bright. The accretion includes diffuse gas and minor and (rare) major mergers. Blue symbols indicate the median rates of a control sample, consisting of galaxies with stellar masses within $\pm 0.2$ dex of the median stellar mass of the submm galaxies. This panel indicates that the submm event was likely triggered by a high rate of gas inflow. **e**, Disc-to-Total (D/T) mass ratio of all galaxies at $z=0$. Orange symbols highlight the D/T ratio of the $z=0$ descendants of the submm galaxies. **f**, Similar to panel **e**, stellar [$\alpha$/Fe](represented by [O/Fe]) of all galaxies at $z=0$, as well as of the submm descendants. Panels **e** and **f** indicate that submm galaxies evolve into elliptical galaxies with typical [$\alpha$/Fe] element ratios. (*Eagle_data*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.16578"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Hints of planet formation signatures in a large-cavity disk \ studied in the AGE-PRO ALMA Large Program

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.16651-b31b1b.svg)](https://arxiv.org/abs/2407.16651)<mark>Appeared on: 2024-07-24</mark> -  _24 pages, 15 figures_

</div>
<div id="authors">

A. Sierra, et al. -- incl., <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** Detecting planet signatures in protoplanetary disks is fundamental to understanding how and where planets form. In this work, we report dust and gas observational hints of planet formation in the disk around 2MASS-J16120668-301027, as part of the ALMA Large Program "AGE-PRO: ALMA survey of Gas Evolution in Protoplanetary disks". The disk was imaged with the Atacama Large Millimeter/submillimeter Array (ALMA) at Band 6 (1.3 mm) in dust continuum emission and four molecular lines: $^{12}$ CO(J=2-1), $^{13}$ CO(J=2-1),  C $^{18}$ O(J=2-1), and $H_2$ CO(J=3 $_{(3,0)}$ -2 $_{(2,0)}$ ).Resolved observations of the dust continuum emission (angular resolution of $\sim 150$ mas, 20 au) show a ring-like structure with a peak at $0.57 ^{\prime \prime}$ (75 au), a deep gap with a minimum at 0.24 $^{\prime \prime}$ (31 au), an inner disk, a bridge connecting the inner disk and the outer ring, along with a spiral arm structure, and a tentative detection (to $3\sigma$ ) of a compact emission at the center of the disk gap, with an estimated dust mass of $\sim 2.7-12.9$ Lunar masses.  We also detected a kinematic kink (not coincident with any dust substructure) through several $^{12}$ CO channel maps (angular resolution $\sim$ 200 mas, 30 au), located at a radius of $\sim 0.875^{\prime \prime}$ (115.6 au).After modeling the $^{12}$ CO velocity rotation around the protostar, we identified a purple tentative rotating-like structure at the kink location with a geometry similar to that of the disk.We discuss potential explanations for the dust and gas substructures observed in the disk, and their potential connection to signatures of planet formation.

</div>

<div id="div_fig1">

<img src="tmp_2407.16651/./Continuum_same_geom.png" alt="Fig11" width="100%"/>

**Figure 11. -** Dust continuum modeling of the disk around J16120. Top left panel: Observed binned visibilities (blue and orange dots, at a bin size of $10^{3}$ and $10^{4} \lambda$, respectively), and visibility model (black curve) of the dust continuum data. Top right panel: Deprojected continuum radial profile from the image plane (blue curve) and from the visibility fit non-convolved model (orange curve). The arrow marks the radial position of the compact emission. Bottom left panel: CLEAN image of the dust continuum data. Bottom middle panel: CLEAN image of the visibility model. Bottom right panel: CLEAN image of the visibility residuals. The iso-contour in the bottom panels is at a 3$\sigma$ level. (*fig:continuum-analysis*)

</div>
<div id="div_fig2">

<img src="tmp_2407.16651/./Residual_map.png" alt="Fig12" width="100%"/>

**Figure 12. -** Spiral arm structure inferred from the disk azimuthal asymmetries. Left panel: Zoom into the residual map in Figure \ref{fig:continuum-analysis}. Black markers are the position of the spiral arm structure, the green solid line is the best fit spiral model, and its extrapolation is shown as a green dashed line. The purple lines around the red knots are 3$\sigma$ iso-contours.
    The blue lines at 0 and 90 degrees are the disk major and minor axis, respectively.
    Right panel: Azimuthal position and deprojected radius of the spiral arm structure (black markers), and best fit model (green solid line). The thin turquoise solid lines represent 1000 random chains from the posterior distribution. (*fig:continuum-residual*)

</div>
<div id="div_fig3">

<img src="tmp_2407.16651/./robust0._CO.png" alt="Fig13" width="100%"/>

**Figure 13. -** Selected $^{12}$CO channel maps of J16120. The white iso-contour is taken from the dust continuum emission in Figure \ref{fig:continuum-analysis}. The synthesized beam and the channel velocity are shown in the bottom-left and top-right corner of each panel, respectively. The color bar is not linear but slightly adjusted to highlight the kink emission. (*fig:CO-channels*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.16651"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bpos}{B_{\mathrm{pos}}}$</div>



<div id="title">

# Magnetic Fields in Massive Star-forming Regions (MagMaR): Unveiling an Hourglass Magnetic Field in G333.46$-$0.16 using ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.16654-b31b1b.svg)](https://arxiv.org/abs/2407.16654)<mark>Appeared on: 2024-07-24</mark> - 

</div>
<div id="authors">

P. Saha, et al. -- incl., <mark>J. Liu</mark>, <mark>S. Li</mark>, <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ The contribution of the magnetic field to the formation of high-mass stars is poorly understood. We report the high-angular resolution ( $\sim0.3^{\prime\prime}$ , 870 au) map of the magnetic field projected on the plane of the sky ( $\bpos$ ) towards the high-mass star forming region G333.46 $-$ 0.16 (G333), obtained with the Atacama Large Millimeter/submillimeter Array (ALMA) at 1.2 mm as part of the Magnetic Fields in Massive Star-forming Regions (MagMaR) survey. The $\bpos$ morphology found in this region is consistent with a canonical "hourglass" which suggest a dynamically important field. This region is fragmented into two protostars separated by $\sim1740$ au. Interestingly, by analysing H $^{13}$ CO $^{+}$ ( $J=3-2$ ) line emission, we find no velocity gradient over the extend of the continuum which is consistent with a strong field. We model the $\bpos$ , obtaining a marginally supercritical mass-to-flux ratio of 1.43, suggesting a initially strongly magnetized environment. Based on the Davis–Chandrasekhar–Fermi method, the magnetic field strength towards G333 is estimated to be $5.7$ mG.The absence of strong rotation and outflows towards the central region of G333 suggests strong magnetic braking, consistent with a highly magnetized environment. Our study shows that despite being a strong regulator, the magnetic energy fails to prevent the process of fragmentation, as revealed by the formation of the two protostars in the central region.

</div>

<div id="div_fig1">

<img src="tmp_2407.16654/./G333.46-0.16_posangle_debias.png" alt="Fig5" width="100%"/>

**Figure 5. -** **(a)** The B$_\mathrm{POS}$ geometry (black line segments) towards G333, obtained after rotating the polarization segments by 90$^{\circ}$, overplotted on ALMA 1.2 mm dust continuum emission. The B$_\mathrm{POS}$ segments have arbitrary length and plotted above the 3$\sigma$($\sigma=29\mu$Jy beam$^{-1}$ for Stokes $Q$ and $U$) level. Contours correspond to dust continuum emission 5, 10, 50, 100, 190, 230, 300 and 340 times $\sigma$($=160 \mu$Jy beam$^{-1}$). The positions of MM1 and MM2 are marked as `+' symbols. The circle in magenta represents the area of analysis. The scale bar and the beamsize of 880 au are displayed on the bottom right and bottom left sides, respectively. The line segments are drawn following the Nyquist sampling (every three pixels each of size $0.05^{\prime\prime}$). **(b)** Magnified view of the area of analysis. Symbols are the same as in panel **(a)**. (*fig:bpos*)

</div>
<div id="div_fig2">

<img src="tmp_2407.16654/./mom0_spw1_H13CO_G333.46_1.png" alt="Fig1" width="100%"/>

**Figure 1. -** **(a)** The moment 0 or integrated intensity map of H$^{13}$CO$^{+}$ line emission. **(b)** The moment 1 or intensity weighted velocity map of the same. The white line segments show the B$_\mathrm{POS}$ geometry and the contours outline the dust continuum emission (same as Figure \ref{fig:bpos}). The positions of MM1 and MM2 are marked as `+' symbols. The scale bar and the spatial resolution are displayed on the bottom right and bottom left sides, respectively, in all panels. (*fig:mom_maps*)

</div>
<div id="div_fig3">

<img src="tmp_2407.16654/./outflow_G333.46_test.png" alt="Fig4" width="100%"/>

**Figure 4. -** Molecular outflows identified in $^{12}$CO $(J=3-2)$ overplotted on our observed 1.2 mm dust continuum emission. The plus symbols mark the positions of MM1 and MM2. The circle in cyan shows the region of analysis. The blueshifted and redshifted outflows are shown in blue and red contours, respectively. The blue contours are drawn with levels 3,5,7,9,12, and 18 times 0.13 Jy beam$^{-1}$. The red contours are drawn with levels 3,5,7,9,11,15,20, and 25 times 0.10 Jy beam$^{-1}$. The possible sources driving the ouflows are marked using green `X' symbols. The red and blue arrows show the directions of the redshifted and blueshifted outflows, respectively. (*fig:outflow*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.16654"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

146  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

8  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
